In [1]:
import distutils


In [2]:
!mlflow --version

mlflow, version 2.15.1


In [4]:
# importing relevant libraries

import pandas as pd
import datetime
import numpy as np

#importing expriment tracking 
import mlflow

#setting tracking uri 

mlflow.set_tracking_uri("http://127.0.0.1:5000")
#mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

2024/08/11 16:59:48 INFO mlflow.tracking.fluent: Experiment with name 'nyc_taxi_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1723395588442, experiment_id='1', last_update_time=1723395588442, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

# Data links 
Jan - 2023 https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Feb 2023  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Mar 2023  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

In [5]:
#reading in the dataset
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet').sample(n = 50000)

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2592680,2,2023-01-27 19:32:24,2023-01-27 19:36:39,2.0,0.64,1.0,N,262,141,1,6.5,2.5,0.5,1.50,0.0,1.0,14.50,2.5,0.0
1746489,1,2023-01-19 16:54:13,2023-01-19 17:07:34,1.0,0.90,1.0,N,90,48,2,10.0,5.0,0.5,0.00,0.0,1.0,16.50,2.5,0.0
1808836,2,2023-01-20 07:18:41,2023-01-20 07:28:01,1.0,2.19,1.0,N,107,162,1,12.1,0.0,0.5,1.90,0.0,1.0,18.00,2.5,0.0
1987014,2,2023-01-21 19:44:18,2023-01-21 19:53:42,1.0,3.18,1.0,N,88,33,1,14.9,0.0,0.5,2.00,0.0,1.0,20.90,2.5,0.0
2244048,1,2023-01-24 16:01:01,2023-01-24 16:29:32,1.0,6.70,1.0,N,137,49,1,30.3,5.0,0.5,7.35,0.0,1.0,44.15,2.5,0.0


In [7]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
# Computing duration 
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)

/tmp/ipykernel_65253/1280888230.py:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['duration'] = df['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)


In [9]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
2592680,2,2023-01-27 19:32:24,2023-01-27 19:36:39,2.0,0.64,1.0,N,262,141,1,6.5,2.5,0.5,1.50,0.0,1.0,14.50,2.5,0.0,4.0
1746489,1,2023-01-19 16:54:13,2023-01-19 17:07:34,1.0,0.90,1.0,N,90,48,2,10.0,5.0,0.5,0.00,0.0,1.0,16.50,2.5,0.0,13.0
1808836,2,2023-01-20 07:18:41,2023-01-20 07:28:01,1.0,2.19,1.0,N,107,162,1,12.1,0.0,0.5,1.90,0.0,1.0,18.00,2.5,0.0,9.0
1987014,2,2023-01-21 19:44:18,2023-01-21 19:53:42,1.0,3.18,1.0,N,88,33,1,14.9,0.0,0.5,2.00,0.0,1.0,20.90,2.5,0.0,9.0
2244048,1,2023-01-24 16:01:01,2023-01-24 16:29:32,1.0,6.70,1.0,N,137,49,1,30.3,5.0,0.5,7.35,0.0,1.0,44.15,2.5,0.0,29.0


In [10]:
# Computing the standard deviation of the trips in Jnauary
np.std(df['duration'])

49.41758817732408

In [11]:
# Computing the percentage of data left after dropping outliers  

df['duration'].shape[0]

50000

In [12]:
# Applying the defined outlier logic
df_trimmed = df[(df['duration'] >= 1 )&(df['duration'] <= 60)]

In [13]:
df_trimmed.shape[0]

49173

In [14]:
#computing the fraction left 
100 - ((df['duration'].shape[0] - df_trimmed.shape[0])/(df['duration'].shape[0]))*100

98.346

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
df_trimmed.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [17]:
categorical = ['PULocationID','DOLocationID']
numerical = ['duration']

In [18]:
# encoding the prescribed columns 
# casting as strings first

df_trimmed['PULocationID'] = df_trimmed['PULocationID'].astype('str')
df_trimmed['DOLocationID'] = df_trimmed['PULocationID'].astype('str')



/tmp/ipykernel_65253/1427150686.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trimmed['PULocationID'] = df_trimmed['PULocationID'].astype('str')
/tmp/ipykernel_65253/1427150686.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trimmed['DOLocationID'] = df_trimmed['PULocationID'].astype('str')


In [19]:
from sklearn.feature_extraction import DictVectorizer

In [20]:
# fitting the encoder and deriving a feature matrix
#df_trimmed = df_trimmed.sample(n = 500000)

# y = df_trimmed.duration

# df_trimmed = df_trimmed[['PULocationID', 'DOLocationID']]
# X = pd.get_dummies(df_trimmed, columns=['PULocationID', 'DOLocationID'])

dv = DictVectorizer()
train_dicts = df_trimmed[['PULocationID', 'DOLocationID']].to_dict(orient = 'records')

In [21]:
X_train = dv.fit_transform(train_dicts)

In [22]:
y_train = df_trimmed['duration'].values

In [23]:
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [24]:
#logging experiment 1 with MLFlow - Linear regression 
with mlflow.start_run():

    mlflow.set_tag("developer", "Abiodun Gbadamosi")

    mlflow.log_param("train_data_path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param("validation_data_path",'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

    model = LinearRegression()
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    
    
    rmse = mean_squared_error(y_train, y_pred, squared = False)
    mlflow.log_metric("rmse", rmse)
print(f"Mean squared error: {rmse:.2f}")

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/08/11 17:04:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run selective-bear-253 at: http://127.0.0.1:5000/#/experiments/1/runs/c47de7a4b7f34df8b589eaa49b5cffa1.
2024/08/11 17:04:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


Mean squared error: 8.53


In [25]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X.sample, y, test_size=0.2, random_state=42)

#logging experiment 2 with MLFlow - XG Boost (USING THE HYPEROPT HYPERPARAMETER TUNING LIBRARY)

In [26]:
# bringing in my validation set
validation = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet').sample(n = 50000)
# Computing duration 
validation['duration'] = validation['tpep_dropoff_datetime'] - validation['tpep_pickup_datetime']
validation['duration'] = validation['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)


# validation  
# casting as strings first
validation = validation[(validation['duration'] >= 1 )&(validation['duration'] <= 60)]
validation['PULocationID'] = validation['PULocationID'].astype('str')
validation['DOLocationID'] = validation['PULocationID'].astype('str')

y_val = validation.duration.values

validation = validation[['PULocationID', 'DOLocationID']].to_dict(orient = 'records')


X_validation = dv.transform(validation)


/tmp/ipykernel_65253/2736253522.py:5: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  validation['duration'] = validation['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)


In [27]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_validation, label=y_val)

In [29]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [30]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.92014                                                                         
[1]	validation-rmse:8.78327                                                                         
[2]	validation-rmse:8.75825                                                                         
[3]	validation-rmse:8.75258                                                                         
[4]	validation-rmse:8.75095                                                                         
[5]	validation-rmse:8.74959                                                                         
  0%|                                                        | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:04:49] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[6]	validation-rmse:8.74885                                                                         
[7]	validation-rmse:8.74847                                                                         
[8]	validation-rmse:8.74775                                                                         
[9]	validation-rmse:8.74779                                                                         
[10]	validation-rmse:8.74686                                                                        
[11]	validation-rmse:8.74647                                                                        
[12]	validation-rmse:8.74589                                                                        
[13]	validation-rmse:8.74554                                                                        
[14]	validation-rmse:8.74571                                                                        
[15]	validation-rmse:8.74535                                                               

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/08/11 17:04:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run hilarious-yak-476 at: http://127.0.0.1:5000/#/experiments/1/runs/2eef1f70d3a14a3b9c5888f05092520d.

2024/08/11 17:04:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



[0]	validation-rmse:9.39683                                                                         
[1]	validation-rmse:9.04007                                                                         
  2%|▌                              | 1/50 [00:05<04:03,  4.97s/trial, best loss: 8.726399536011288]

...

[0]	validation-rmse:10.03222                                                                        
[1]	validation-rmse:9.92555                                                                         
 98%|█████████████████████████████▍| 49/50 [04:40<00:08,  8.10s/trial, best loss: 8.711141213612093]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:09:29] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:9.82798                                                                         
[3]	validation-rmse:9.73879                                                                         
[4]	validation-rmse:9.65723                                                                         
[5]	validation-rmse:9.58267                                                                         
[6]	validation-rmse:9.51443                                                                         
[7]	validation-rmse:9.45231                                                                         
[8]	validation-rmse:9.39523                                                                         
[9]	validation-rmse:9.34331                                                                         
[10]	validation-rmse:9.29575                                                                        
[11]	validation-rmse:9.25144                                                               

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/08/11 17:09:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run intelligent-mole-418 at: http://127.0.0.1:5000/#/experiments/1/runs/f5518be57548494f8913327c4fa6f4de.

2024/08/11 17:09:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.



100%|██████████████████████████████| 50/50 [04:54<00:00,  5.89s/trial, best loss: 8.711141213612093]


In [31]:
mlflow.xgboost.autolog(disable=True)

In [32]:
#repeating the training again with the best model and log params 
import pickle
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_validation, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:9.92972
[1]	validation-rmse:9.74524
[2]	validation-rmse:9.59049
[3]	validation-rmse:9.46128
[4]	validation-rmse:9.35217
[5]	validation-rmse:9.26192
[6]	validation-rmse:9.18560
[7]	validation-rmse:9.12160
[8]	validation-rmse:9.06863
[9]	validation-rmse:9.02335
[10]	validation-rmse:8.98658
[11]	validation-rmse:8.95481
[12]	validation-rmse:8.92896


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:15:54] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[13]	validation-rmse:8.90681
[14]	validation-rmse:8.88567
[15]	validation-rmse:8.86987
[16]	validation-rmse:8.85667
[17]	validation-rmse:8.84327
[18]	validation-rmse:8.83350
[19]	validation-rmse:8.82516
[20]	validation-rmse:8.81623
[21]	validation-rmse:8.80990
[22]	validation-rmse:8.80444
[23]	validation-rmse:8.79835
[24]	validation-rmse:8.79425
[25]	validation-rmse:8.79078
[26]	validation-rmse:8.78753
[27]	validation-rmse:8.78374
[28]	validation-rmse:8.78138
[29]	validation-rmse:8.77896
[30]	validation-rmse:8.77712
[31]	validation-rmse:8.77440
[32]	validation-rmse:8.77285
[33]	validation-rmse:8.77128
[34]	validation-rmse:8.77026
[35]	validation-rmse:8.76831
[36]	validation-rmse:8.76636
[37]	validation-rmse:8.76550
[38]	validation-rmse:8.76400
[39]	validation-rmse:8.76330
[40]	validation-rmse:8.76182
[41]	validation-rmse:8.76134
[42]	validation-rmse:8.76003
[43]	validation-rmse:8.75896
[44]	validation-rmse:8.75838
[45]	validation-rmse:8.75811
[46]	validation-rmse:8.75729
[47]	validatio

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:16:13] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  war

In [33]:
#Repeating the training for four other models 
# to use the mlflow modle registry

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [34]:
#!pip install scikit-learn==1.0.0


In [35]:
mlflow.sklearn.autolog()

In [45]:
y_val.shape , y_pred.shape

((50000,), (49133,))

In [46]:
# bringing in my validation set
#reinitializing because I don't need the transformed values anymore
validation = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet').sample(n = 50000)
# Computing duration 
validation['duration'] = validation['tpep_dropoff_datetime'] - validation['tpep_pickup_datetime']
validation['duration'] = validation['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)

#y_val = validation.duration.values
#trimming the validation values also 
y_val = validation[(validation['duration'] >= 1 )&(validation['duration'] <= 60)].duration.values

#validation = validation[['PULocationID', 'DOLocationID']]

#changed the line above to the code below to account for shape mismatch
# preprocessing the validation dataset the same way the training dataset was preprocessed
validation_trimmed = validation[(validation['duration'] >= 1 )&(validation['duration'] <= 60)]
validation_dicts = validation_trimmed[['PULocationID','DOLocationID']].to_dict(orient = 'records')
validation = dv.transform(validation_dicts)


for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.set_tag("developer", "Abiodun Gbadamosi")

     #   mlflow.log_param("train_data_path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
     #   mlflow.log_param("validation_data_path",'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(validation)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

/tmp/ipykernel_65253/3171364386.py:6: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  validation['duration'] = validation['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)
2024/08/11 17:34:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/08/11 17:34:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run sedate-hen-930 at: http://127.0.0.1:5000/#/experiments/1/runs/0e2e5ac0f90845b18d3a50b49553af30.
2024/08/11 17:34:29 INFO mlflow.tracking._tracki

In [53]:
#PROMOTING THE BEST MODEL FROM THE MLFlow Model Registry

from mlflow.tracking import MlflowClient 

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [55]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1723395588442, experiment_id='1', last_update_time=1723395588442, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723393822766, experiment_id='0', last_update_time=1723393822766, lifecycle_stage='active', name='Default', tags={}>]

In [58]:
from mlflow.entities import ViewType

In [61]:
runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5, 
    order_by = ["metrics.rmse ASC"]
)

for run in runs: 
    print(f"run id : {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id : c47de7a4b7f34df8b589eaa49b5cffa1, rmse: 8.5298
run id : e3c4f9b5f4614cdf9657baaf9dc20a9a, rmse: 8.7111
run id : 45f274005d974445b55105d20b6462c5, rmse: 8.7112
run id : 6770e4a1373e42568406de51040ffc9c, rmse: 8.7139
run id : 219ca1b396bf4a4ab56dcde7300cc619, rmse: 8.7139


In [62]:
# registered model 

import mlflow 

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [69]:
#promoting a model to the model registry 
run_id = "c47de7a4b7f34df8b589eaa49b5cffa1"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri = model_uri , name ="best_linear_regression_model") 

Successfully registered model 'best_linear_regression_model'.
2024/08/11 17:47:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best_linear_regression_model, version 1
Created version '1' of model 'best_linear_regression_model'.


<ModelVersion: aliases=[], creation_timestamp=1723398479913, current_stage='None', description='', last_updated_timestamp=1723398479913, name='best_linear_regression_model', run_id='c47de7a4b7f34df8b589eaa49b5cffa1', run_link='', source='mlflow-artifacts:/1/c47de7a4b7f34df8b589eaa49b5cffa1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [77]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1723398479848, description='', last_updated_timestamp=1723398479913, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1723398479913, current_stage='None', description='', last_updated_timestamp=1723398479913, name='best_linear_regression_model', run_id='c47de7a4b7f34df8b589eaa49b5cffa1', run_link='', source='mlflow-artifacts:/1/c47de7a4b7f34df8b589eaa49b5cffa1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='best_linear_regression_model', tags={}>]

In [79]:
client.get_registered_model(name = 'best_linear_regression_model')

<RegisteredModel: aliases={}, creation_timestamp=1723398479848, description='', last_updated_timestamp=1723398479913, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1723398479913, current_stage='None', description='', last_updated_timestamp=1723398479913, name='best_linear_regression_model', run_id='c47de7a4b7f34df8b589eaa49b5cffa1', run_link='', source='mlflow-artifacts:/1/c47de7a4b7f34df8b589eaa49b5cffa1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='best_linear_regression_model', tags={}>

In [80]:
# getting the versions of a latest model
model_name = "best_linear_regression_model"
latest_versions = client.get_latest_versions(name = model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    

version: 1, stage: None


/tmp/ipykernel_65253/3364933.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


In [81]:
#transition the stage of a registered model 
model_version = 1 
new_stage = "Staging" 

client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = False
)

/tmp/ipykernel_65253/3894736172.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1723398479913, current_stage='Staging', description='', last_updated_timestamp=1723398746683, name='best_linear_regression_model', run_id='c47de7a4b7f34df8b589eaa49b5cffa1', run_link='', source='mlflow-artifacts:/1/c47de7a4b7f34df8b589eaa49b5cffa1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [83]:
#adding a description or annotating model transitions 
from datetime import datetime 
date = datetime.today().date()

client.update_model_version(
    name = model_name, 
    version = model_version, 
    description = f"The model version{model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1723398479913, current_stage='Staging', description='The model version1 was transitioned to Staging on 2024-08-11', last_updated_timestamp=1723398787713, name='best_linear_regression_model', run_id='c47de7a4b7f34df8b589eaa49b5cffa1', run_link='', source='mlflow-artifacts:/1/c47de7a4b7f34df8b589eaa49b5cffa1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [103]:
#Simulating a deployment engineer's workflow

from sklearn.metrics import mean_squared_error
import pandas as pd

train_filepath = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
validation_filepath = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

def read_dataframe(filename):
    df = pd.read_parquet(train_filepath)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    #converting the Pickup and Dropoff locations to 
    #string before concatenation
    df['PU_DO'] = df['PULocationID'].astype('str') + '_' + df['DOLocationID'].astype('str')
    categorical = ['PU_DO']
    numerical = ['duration']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}



In [104]:
# get the dataframe 
df = pd.read_parquet(validation_filepath)

In [105]:
# Getitng the rinid for the best linear regression model
run_id

'c47de7a4b7f34df8b589eaa49b5cffa1'

In [91]:
# Probable issue with too many github redirects when tryin to download the model
#loading my dictionary vectorizer nd placing it in my root directory  
client.download_artifacts(run_id = run_id , path = 'preprocessor' , dst_path = '.' )



In [106]:
#loading the (local file) with pickle using the relative path
with open("/workspaces/mlops-zoomcamp/02-experiment-tracking/models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'PU_DO'],
      dtype='object')

In [114]:
# Computing duration 
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)

/tmp/ipykernel_65253/1280888230.py:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['duration'] = df['duration'].apply(lambda x : np.round(x.total_seconds()/60),2)


In [116]:
X_test = preprocess(df, dv)


target = "duration"
y_test = df[target].values

In [ ]:
#%time test_model(name = model_name , stage = "Production", X_TEST , y_test)

In [120]:
model_name

'best_linear_regression_model'

In [122]:
#testing the model in staging as well 
# same codespaces error with downloading
%time test_model(name = model_name , stage = "Staging", X_test=X_test , y_test=y_test)


In [123]:
model_version

1

In [ ]:
#incase I want to transition the staging model to production 

client.transition_model_version_stage(
    name = model_name,
    version = model_version , 
    stage = "Production", 
    archive_existing_versions = True
    
)